In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [2]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [3]:
# !pip install git+https://github.com/huggingface/transformers

In [4]:
!ls -lah /root/.cache/huggingface/hub/

total 81M
drwxr-xr-x 10 root root 4.0K Oct 26 13:21 .
drwxr-xr-x  5 root root   61 Oct 24 15:26 ..
drwxr-xr-x  6 root root   65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x  6 root root   65 Oct 24 18:26 models--bigscience--bloom-7b1
drwxr-xr-x  6 root root   65 Aug 30 22:52 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x  6 root root   65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x  6 root root   65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x  2 root root    6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x  6 root root   65 Oct 26 13:21 models--tiiuae--falcon-7b
drwxr-xr-x  5 root root   48 Oct 26 09:36 models--ysw96--my_awesome_peft_model
-rw-------  1 root root  80M Oct 26 08:22 tmpd_8zze2t
-rw-------  1 root root    0 Oct 24 11:18 tmpxe5e_q0g
-rw-r--r--  1 root root    1 Jul 26 16:10 version.txt


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [6]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [7]:
!nvidia-smi -L

GPU 0: NVIDIA A40 (UUID: GPU-84d1dbbd-b6a7-03b8-feb8-3ecfdf6b7b02)


In [8]:
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq bitsandbytes==0.39.0 --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
# !pip install -qqq datasets==2.12.0 --progress-bar off
# !pip install -qqq loralib==0.1.1 --progress-bar off
# !pip install -qqq einops==0.6.1 --progress-bar off

In [9]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-10-27 15:21:02.334966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 15:21:03.050116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
import torch
torch.cuda.device_count()

1

In [11]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [12]:
MODEL_NAME = "FlagAlpha/Llama2-Chinese-7b-Chat"
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # trust_remote_code=True,
    # local_files_only=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [42]:
!nvidia-smi

Fri Oct 27 15:26:19 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   28C    P0              68W / 300W |  37376MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [15]:
print_trainable_parameters(model)

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


In [16]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [17]:
# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


## Inference Before Training

In [22]:
prompt = f"""
: 请问如何到达指定的工作站点？
:
""".strip()
print(prompt)

: 请问如何到达指定的工作站点？
:


In [23]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [24]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.7,
  "top_p": 0.7
}

In [25]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: 请问如何到达指定的工作站点？
: 我需要到达某个工作站点，但我不知道该站点的具体位置。请问您能提供一些帮助？

CPU times: user 5.66 s, sys: 25.4 ms, total: 5.69 s
Wall time: 5.7 s


In [30]:
with open("./data/qa_dataset_cn.json") as json_file:
    data = json.load(json_file)

In [31]:
pprint(data[0], sort_dicts=False)

{'question': '如何跳轉到指定工站？',
 'answer': '需要登錄管理員帳號, 服務中心後臺->進度管控->進度維護, 進入【保修項目】分頁, 勾選對應產品序號, '
           '下拉【維修操作】點擊【進度維護】, 選擇【跳轉至指定工站】即可進行跳轉.'}


In [32]:
pprint(data[1], sort_dicts=False)

{'question': '所收返修品和保修申請內容不符如何處理？',
 'answer': '服務中心後台->貨物收發->收發管理, 在【待收貨】分頁進行收貨時, 如若出現貨不符實的情況, 可選取該產品點擊【刪除】, '
           '該產品資訊將會返回給客戶確認, 客戶可重新提交申請.'}


In [33]:
# data = load_dataset("json", data_files="./data/qa_dataset.json")

In [35]:
import pandas as pd
data = pd.read_csv("./data/warreconn_data_cn.csv")
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [36]:
dataset[0]

{'question': '如何跳轉到指定工站？',
 'answer': '需要登錄管理員帳號, 服務中心後臺->進度管控->進度維護, 進入【保修項目】分頁, 勾選對應產品序號, 下拉【維修操作】點擊【進度維護】, 選擇【跳轉至指定工站】即可進行跳轉.'}

In [37]:
def generate_prompt(data_point):
    return f"""
: {data_point["question"]}
: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [38]:
train_data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [39]:
train_data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 6
})

## Training

In [40]:
OUTPUT_DIR = "experiments"

In [41]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [43]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

In [44]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [45]:
%%time
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.359600
2,2.385400
3,2.159200
4,1.858400
5,1.530900
6,1.213500
7,0.954600
8,0.818400
9,0.484700
10,0.351300


CPU times: user 2min 58s, sys: 23.9 s, total: 3min 22s
Wall time: 3min 22s


TrainOutput(global_step=80, training_loss=0.19450877126073465, metrics={'train_runtime': 202.1692, 'train_samples_per_second': 1.583, 'train_steps_per_second': 0.396, 'total_flos': 2051509370806272.0, 'train_loss': 0.19450877126073465, 'epoch': 53.33})

## Save the trained model

In [46]:
model.save_pretrained("llama2_cn_trained-model")

In [47]:
# from huggingface_hub import notebook_login

# notebook_login()

In [48]:
# peft_model_name = "llama2_cn_trained-model"
# model.push_to_hub(peft_model_name)

## Load Trained Model

In [49]:
# PEFT_MODEL = f"ysw96/{peft_model_name}"
PEFT_MODEL = f"./llama2_cn_trained-model"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    # trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Inference

In [50]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [51]:
DEVICE = "cuda:0"

In [63]:
%%time
prompt = f"""
: 如何跳轉到指定工站？
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: 如何跳轉到指定工站？
: 需要登錄管理員帳號, 服務中心後臺->進度管控->進度維護, 進入【保修項目】分頁, 勾選對應產品序號, 下拉【維修操作】點擊【進度維護】, 選擇【跳轉至指定工站】即可進行跳轉.

CPU times: user 13 s, sys: 25.9 ms, total: 13 s
Wall time: 13 s


In [53]:
def generate_response(question: str) -> str:
    prompt = f"""
: {question}
:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [64]:
prompt = "如何跳轉到指定工站？"
print(generate_response(prompt))

如何跳轉到指定工站？
: 需要登錄管理員帳號, 服務中心後臺->進度管控->進度維護, 進入【保修項目】分頁, 勾選對應產品序號, 下拉【維修操作】點擊【進度維護】, 選擇【跳轉至指定工站】即可進行跳轉.


In [55]:
prompt = "所收返修品和保修申請內容不符如何處理？"
print(generate_response(prompt))

所收返修品和保修申請內容不符如何處理？
: 服務中心後台->貨物收發->收發管理, 在【待收貨】分頁進行收貨時, 如若出現貨不符實的情況, 可選取該產品點擊【刪除】, 該產品資訊將會返回給客戶確認, 客戶可重新提交申請.


In [57]:
%%time
prompt = "針對工站列印標籤, 服務中心可聯繫廠家端進行修改, 廠家維修服務後台->工程支援->工站列印, 可選擇對應列印範本對標籤內容進行修改."
print(generate_response(prompt))

針對工站列印標籤, 服務中心可聯繫廠家端進行修改, 廠家維修服務後台->工程支援->工站列印, 可選擇對應列印範本對標籤內容進行修改.
: 客戶可在工站列印標籤頁面, 勾選對應列印範本, 下拉【範本對標籤內容修改】點擊【對標籤內容進行修改】, 可選擇對應列印範本對標籤內容進行修改.
: 服務中心可聯繫廠家維修服務後台, 選擇【工站列印】分頁, 可勾選對應列印範
CPU times: user 18 s, sys: 120 ms, total: 18.1 s
Wall time: 18.2 s


In [61]:
%%time
prompt = "客戶可以在哪裡查看工站作業記錄？"
print(generate_response(prompt))

客戶可以在哪裡查看工站作業記錄？
: 客戶後台->保修作業->服務進度, 在保修項目頁面, 可查看已送修SN的進度詳情頁, 服務進度下的工作記錄詳情, 可查看具體工站的作業情況.
CPU times: user 8.49 s, sys: 26.6 ms, total: 8.51 s
Wall time: 8.53 s


In [62]:
%%time
prompt = "客戶在哪裡可以看到單筆SN的替換備品詳情？"
print(generate_response(prompt))

客戶在哪裡可以看到單筆SN的替換備品詳情？
: 客戶後台->保修作業->服務進度, 在保修項目頁面, 可查看已送修SN的進度詳情頁, 服務進度下的替換備品詳情, 可查看該筆SN在工站作業時的備品替換情況.
CPU times: user 10 s, sys: 28.2 ms, total: 10 s
Wall time: 10 s


In [60]:
%%time
prompt = "如果消費者沒收到郵寄標籤, 系統可以在哪裡下載？"
print(generate_response(prompt))

如果消費者沒收到郵寄標籤, 系統可以在哪裡下載？
: 客服可在系統篩選下載郵寄標籤, 在貨物收發處, 貨物記錄頁面, 可通過消費者電話、郵箱等消費者信息及產品序列號、物流單號等條件進行篩選, 選定訂單點擊下載按鈕, 可對郵件標籤進行下載列印.
CPU times: user 15.3 s, sys: 112 ms, total: 15.4 s
Wall time: 15.4 s


## Similar question

In [56]:
%%time
prompt = "實際所收貨物和保修單不相符怎麼辦？"
print(generate_response(prompt)) #Q2

實際所收貨物和保修單不相符怎麼辦？
: 客服可在貨物收發處、保修申請處等地方進行跳轉、聯繫客戶端進行貨物收發和保修作業, 客服可在貨物收發處進行跳轉, 如若貨不符實、貨不符單, 客服可在保修申請處進行申請, 服務中心可在貨物收發處進行跳轉, 客服可在保修申請處進行申請, 服務中
CPU times: user 18 s, sys: 41.7 ms, total: 18.1 s
Wall time: 18.1 s
